# Benchmarking XGBoost on GPU

In [1]:
import numpy as np
import pandas as pd

import xgboost as xgb

import sklearn
from sklearn.model_selection import train_test_split

from sklearn.datasets import fetch_openml

In [2]:
print(f'numpy: {np.__version__}')
print(f'pandas: {pd.__version__}')
print(f'Scikit-learn: {sklearn.__version__}')
print(f'XGBoost: {xgb.__version__}')

numpy: 1.26.0
pandas: 2.1.2
Scikit-learn: 1.3.2
XGBoost: 1.7.6


In [3]:
random_state = 1

In [4]:
covtyp = fetch_openml(
    name='covertype',
    version=4,
    parser='auto'
)

In [5]:
data_features = covtyp.data

In [6]:
data_features.shape

(581012, 54)

In [7]:
data_target = covtyp.target
data_target.shape

(581012,)

In [8]:
! nvidia-smi

Fri Nov 10 17:05:44 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.01              Driver Version: 546.01       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2070        On  | 00000000:01:00.0  On |                  N/A |
| 31%   37C    P5              40W / 175W |   1256MiB /  8192MiB |     38%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
df = pd.concat([data_features, data_target], axis=1)

In [10]:
df.memory_usage(index=True).sum()

72632440

In [11]:
features = data_features.columns

target = 'class'

In [12]:
df[target].value_counts()

class
2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: count, dtype: int64

In [13]:
df.dtypes

Elevation                                int64
Aspect                                   int64
Slope                                    int64
Horizontal_Distance_To_Hydrology         int64
Vertical_Distance_To_Hydrology           int64
Horizontal_Distance_To_Roadways          int64
Hillshade_9am                            int64
Hillshade_Noon                           int64
Hillshade_3pm                            int64
Horizontal_Distance_To_Fire_Points       int64
Wilderness_Area1                      category
Wilderness_Area2                      category
Wilderness_Area3                      category
Wilderness_Area4                      category
Soil_Type1                            category
Soil_Type2                            category
Soil_Type3                            category
Soil_Type4                            category
Soil_Type5                            category
Soil_Type6                            category
Soil_Type7                            category
Soil_Type8   

In [14]:
for column in df.columns:
    df[column] = pd.to_numeric(df[column])

In [15]:
df[target] = df[target] - 1

In [16]:
df.memory_usage(index=True).sum()

255645408

In [17]:
df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,class
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,4
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,4
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,1
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,1
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,2
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,2
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,2
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,2


In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    df[features],
    df[target],
    train_size=0.85,
    test_size=0.15,
    random_state=random_state,
)

In [19]:
%%capture
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [20]:
! nvidia-smi

Fri Nov 10 17:06:00 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.01              Driver Version: 546.01       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2070        On  | 00000000:01:00.0  On |                  N/A |
| 31%   38C    P2              59W / 175W |   1348MiB /  8192MiB |      8%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [21]:
import time
#setting tree and tree depth
num_round = 50
maxdepth = 8
param = {
  'colsample_bylevel': 1,
  'colsample_bytree': 1,
  'gamma': 0,
  'learning_rate': 0.1, 
  'random_state': 1010,
  'objective': 'multi:softmax', 
  'num_class': 7, 
}

In [22]:
param['tree_method'] = 'hist'
param['grow_policy'] = 'depthwise'
param['max_depth'] = maxdepth
param['max_leaves'] = 0
param['verbosity'] = 0
cpu_result = {} 
start_time = time.time()
# Training with the above parameters
xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=cpu_result, verbose_eval=20)

print("CPU Training Time: %s seconds" % (str(time.time() - start_time)))

[0]	test-mlogloss:1.74085
[20]	test-mlogloss:0.63436
[40]	test-mlogloss:0.46410
[49]	test-mlogloss:0.42896
CPU Training Time: 60.95507597923279 seconds


In [24]:
param['tree_method'] = 'gpu_hist'
param['grow_policy'] = 'depthwise'
param['max_depth'] = maxdepth
param['max_leaves'] = 0
param['verbosity'] = 0
param['gpu_id'] = 0
param['updater'] = 'grow_gpu_hist'
param['predictor'] = 'gpu_predictor'

gpu_result = {} 
start_time = time.time()
# Training with the above parameters
xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=gpu_result, verbose_eval=20)

print("GPU Training Time: %s seconds" % (str(time.time() - start_time)))

[0]	test-mlogloss:1.74093
[20]	test-mlogloss:0.63320
[40]	test-mlogloss:0.46331
[49]	test-mlogloss:0.42811
GPU Training Time: 6.380246639251709 seconds


In [25]:
! nvidia-smi

Fri Nov 10 17:10:30 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.01              Driver Version: 546.01       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2070        On  | 00000000:01:00.0  On |                  N/A |
| 32%   37C    P8              40W / 175W |   1818MiB /  8192MiB |     25%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--